In [3]:
import pandas as pd

In [4]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Input
from keras.utils import to_categorical

In [5]:
from keras.layers import Conv2D # to add convolutional layers
from keras.layers import MaxPooling2D # to add pooling layers
from keras.layers import Flatten # to flatten data for fully connected layers

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
!pip install scikit-learn

In [8]:
concrete = pd.read_csv('/home/kronos/Downloads/concrete_data.csv')

In [9]:
concrete_data_columns = concrete.columns

In [10]:
predictors = concrete[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete['Strength']

In [11]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [12]:
n_cols = predictors_norm.shape[1] # number of predictors

In [13]:
class MyModel(keras.Model):
    def __init__(self):
        super().__init__()
        self.dense1 = keras.layers.Dense(50, activation="relu")
        self.dense2 = keras.layers.Dense(1)
        
    def build(self, input_shape):
        self.dense1.build(input_shape)
        self.dense2.build((input_shape[0], 50))  # Output of dense1 is 50
        super().build(input_shape)
        
    def call(self, inputs):
        x = self.dense1(inputs)
        return self.dense2(x)

In [14]:
model = MyModel()
model.build(input_shape=(None, predictors_norm.shape[1]))  # Pass the input shape
model.summary()

I0000 00:00:1734586204.027224   24789 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1734586204.065830   24789 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1734586204.066046   24789 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1734586204.067424   24789 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

Model: "my_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 50)             │           450 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 501 (1.96 KB)

 Trainable params: 501 (1.96 KB)

 Non-trainable params: 0 (0.00 B)

In [56]:
model.compile(optimizer="adam", loss="mse", metrics=["mae"])

In [57]:
model.fit(predictors_norm, target, validation_split=0.3, epochs=100, verbose=2)

Epoch 1/100


W0000 00:00:1734586152.864022   23180 gpu_backend_lib.cc:631] libdevice is required by this HLO module but was not found at ./libdevice.10.bc
2024-12-18 23:29:12.865781: W tensorflow/core/framework/op_kernel.cc:1840] OP_REQUIRES failed at xla_ops.cc:577 : INTERNAL: libdevice not found at ./libdevice.10.bc


InternalError: Graph execution error:

Detected at node StatefulPartitionedCall defined at (most recent call last):
  File "/home/kronos/anaconda3/envs/tensorflow_env/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/home/kronos/anaconda3/envs/tensorflow_env/lib/python3.10/runpy.py", line 86, in _run_code

  File "/home/kronos/anaconda3/envs/tensorflow_env/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/home/kronos/anaconda3/envs/tensorflow_env/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/kronos/anaconda3/envs/tensorflow_env/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/kronos/anaconda3/envs/tensorflow_env/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/home/kronos/anaconda3/envs/tensorflow_env/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/home/kronos/anaconda3

In [27]:
!nvidia-smi       # To check NVIDIA GPU and driver status


Wed Dec 18 23:25:44 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.120                Driver Version: 550.120        CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4050 ...    Off |   00000000:01:00.0 Off |                  N/A |
| N/A   45C    P8              1W /   60W |    4378MiB /   6141MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [43]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  1
